In [4]:
pip install pyLDAvis

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached pyLDAvis-3.4.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached numexpr-2.10.0-cp311-cp311-win_amd64.whl.metadata (8.1 kB)
  Using cached funcy-2.0-py2.py3-none-any.whl.metadata (5.9 kB)
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -----------------------------------

In [2]:
import gensim
from gensim import corpora
import re
import pandas as pd
import nltk
import string
from nltk.corpus import stopwords
import glob
from pprint import pprint
nltk.download('punkt')
nltk.download('stopwords')
indonesian_stopwords = set(stopwords.words('indonesian'))
english_stopwords = set(stopwords.words('english'))
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TEMP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TEMP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
!pip list

Package                      Version
---------------------------- ---------
absl-py                      2.1.0
archspec                     0.2.3
asttokens                    2.4.1
astunparse                   1.6.3
boltons                      23.0.0
Brotli                       1.0.9
certifi                      2024.2.2
cffi                         1.16.0
charset-normalizer           2.0.4
click                        8.1.7
colorama                     0.4.6
comm                         0.2.2
conda                        24.4.0
conda-content-trust          0.2.0
conda-libmamba-solver        23.12.0
conda-package-handling       2.2.0
conda_package_streaming      0.9.0
contourpy                    1.2.1
cryptography                 41.0.7
cycler                       0.12.1
debugpy                      1.8.1
decorator                    5.1.1
dill                         0.3.8
distro                       1.8.0
exceptiongroup               1.2.0
executing                    2.0.1
file

In [3]:
df = pd.read_csv('../Dataset/Testing/Clean_testing_data_nostem.csv')

folder_path = '../Dataset/Testing/Raw'
file_pattern = '*.csv'
file_list = glob.glob(folder_path + file_pattern)

dfs = []

for file in file_list:
    df = pd.read_csv(file)
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df.head()

In [5]:
def fix_contractions_id(text):
    with open('../Dataset/kamus_singkatan.csv', 'r', encoding='utf-8') as file:
        lines = file.readlines()

    data = [line.strip().split('\t') for line in lines]
    data_singkatan = pd.DataFrame(data, columns=['Kata', 'Asli'])

    # Ubah DataFrame menjadi kamus
    kontraksi_dict = dict(zip(data_singkatan['Kata'], data_singkatan['Asli']))
    words = str(text).split()
    expanded_text = [kontraksi_dict[word] if word in kontraksi_dict else word for word in words]

    return ' '.join(expanded_text)

def case_folding(tweet):
    return str(tweet).lower()

def clean_tweet(tweet):
    tweet = str(tweet).lower()
    tweet = re.compile('https?://\S+|www\.\S+').sub(r'', tweet) # Remove hyperlinks

    if tweet.startswith("rt"): # Remove retweets (repetitions)
        i = tweet.find(':')
        if i != -1:  
            tweet = tweet[i+2:]
    
    tweet = re.compile('@[^\s]+').sub(r'', tweet) # Mentions 
    tweet = re.compile(r'#([^\s]+)').sub(r'\1', tweet) # Remove hashtags
    tweet = re.sub('@', 'at', tweet)
    tweet  = ''.join([char for char in tweet if char not in string.punctuation]) #Remove punctuation characters
    tweet = re.compile('[^A-Za-z]').sub(r' ', tweet) # Remove any other non-alphabet characters
    # tweet = ' '.join([w for w in tweet.split() if w not in indonesian_stopwords]) #Remove stop words  
    # tweet = ' '.join([w for w in tweet.split() if w not in english_stopwords]) #Remove stop words
    
    return tweet

def remove_single_letter_words(text):
    text = re.sub(r'\b\w\b', '', text)
    text = ' '.join([w for w in text.split() if w not in indonesian_stopwords]) #Remove stop words  
    
    hapus = ['id','amp','deh','tanyakanrl','sih','na','si','rj','lc','ar','oe','al','sm','ri','en','ar','mc','vt','rob','ny','dc','az','va','mkmk','nya','do','ye','dtype','um','object','length','the','is', 'in', 'of', 'for']
    words = text.split()

    # Memfilter kata-kata yang tidak ada dalam array yang akan dihapus
    kata_kata_tanpa_kata_yang_dihapus = [kata for kata in words if kata not in hapus]

    # Menggabungkan kata-kata yang tersisa menjadi kalimat baru
    kalimat_tanpa_kata_yang_dihapus = ' '.join(kata_kata_tanpa_kata_yang_dihapus)
    return kalimat_tanpa_kata_yang_dihapus



<>:20: DeprecationWarning: invalid escape sequence '\S'
<>:27: DeprecationWarning: invalid escape sequence '\s'
<>:20: DeprecationWarning: invalid escape sequence '\S'
<>:27: DeprecationWarning: invalid escape sequence '\s'
C:\Users\TEMP\AppData\Local\Temp\ipykernel_17544\491285358.py:20: DeprecationWarning: invalid escape sequence '\S'
  tweet = re.compile('https?://\S+|www\.\S+').sub(r'', tweet) # Remove hyperlinks
C:\Users\TEMP\AppData\Local\Temp\ipykernel_17544\491285358.py:27: DeprecationWarning: invalid escape sequence '\s'
  tweet = re.compile('@[^\s]+').sub(r'', tweet) # Mentions


In [6]:
data = df['full_text']
display(data)

print('case folding')
data = data.apply(case_folding)
display(data)

print('cleaning')
data = data.apply(clean_tweet)
display(data)

print('normalization')
data = data.apply(fix_contractions_id)
display(data)

print('stopword')
data = data.apply(remove_single_letter_words)
display(data)

0      ['match', 'lavani', 'vs', 'pbs', 'putar', 'non...
1      ['kasih', 'solusi', 'solusi', 're', 'upload', ...
2                     ['nyiarin', 'indo', 'vs', 'usbek']
3      ['kecewa', 'bayar', 'nonton', 'tim', 'uber', '...
4                              ['oh', 'court', 'kamera']
                             ...                        
926    ['lebaran', 'nobar', 'bareng', 'sayang', 'mimi...
927    ['kanaya', 'ika', 'dihardjo', 'support', 'syst...
928    ['pintu', 'berkah', 'special', 'ramadhan', 'so...
929    ['kisah', 'special', 'ramadhan', 'sore', 'pkl'...
930    ['aliza', 'naura', 'ayu', 'masak', 'kina', 'ba...
Name: full_text, Length: 931, dtype: object

case folding


0      ['match', 'lavani', 'vs', 'pbs', 'putar', 'non...
1      ['kasih', 'solusi', 'solusi', 're', 'upload', ...
2                     ['nyiarin', 'indo', 'vs', 'usbek']
3      ['kecewa', 'bayar', 'nonton', 'tim', 'uber', '...
4                              ['oh', 'court', 'kamera']
                             ...                        
926    ['lebaran', 'nobar', 'bareng', 'sayang', 'mimi...
927    ['kanaya', 'ika', 'dihardjo', 'support', 'syst...
928    ['pintu', 'berkah', 'special', 'ramadhan', 'so...
929    ['kisah', 'special', 'ramadhan', 'sore', 'pkl'...
930    ['aliza', 'naura', 'ayu', 'masak', 'kina', 'ba...
Name: full_text, Length: 931, dtype: object

cleaning


0       match lavani vs pbs putar nonton ulang aksi beik
1      kasih solusi solusi re upload match proliga ke...
2                                  nyiarin indo vs usbek
3                      kecewa bayar nonton tim uber main
4                                        oh court kamera
                             ...                        
926    lebaran nobar bareng sayang mimin gudang tonto...
927    kanaya ika dihardjo support system aliza coba ...
928    pintu berkah special ramadhan sore pkl wib sak...
929    kisah special ramadhan sore pkl wib saksi leng...
930    aliza naura ayu masak kina bahagia zero teuku ...
Name: full_text, Length: 931, dtype: object

normalization


C:\Users\TEMP\AppData\Local\Temp\ipykernel_17544\491285358.py:20: DeprecationWarning: invalid escape sequence '\S'
  tweet = re.compile('https?://\S+|www\.\S+').sub(r'', tweet) # Remove hyperlinks
C:\Users\TEMP\AppData\Local\Temp\ipykernel_17544\491285358.py:27: DeprecationWarning: invalid escape sequence '\s'
  tweet = re.compile('@[^\s]+').sub(r'', tweet) # Mentions


ValueError: 2 columns passed, passed data had 1 columns

In [26]:
data.to_csv('all_data.csv')

In [8]:

tokenized_documents = [doc.split() for doc in data]
# print(tokenized_documents)

# Membuat kamus (dictionary)
dictionary = corpora.Dictionary(tokenized_documents)
# print(dictionary)

# Membuat corpus (representasi vektor dari dokumen)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]
# print(corpus)

# Clusstering with LDA
lda_model = gensim.models.LdaModel(corpus, num_topics=20, id2word=dictionary, passes=20)

topics = lda_model.show_topics(num_topics=20, log=False, formatted=False)

        
coherencemodel = CoherenceModel(model=lda_model, texts=tokenized_documents, dictionary=dictionary, coherence='c_v')
pprint(lda_model.print_topics())
print("Coherence",coherencemodel.get_coherence())

lda_display = gensimvis.prepare(lda_model, corpus, dictionary, sort_topics=False)

pyLDAvis.display(lda_display)


[(0,
  '0.023*"vidio" + 0.021*"sctv" + 0.018*"nyanyi" + 0.018*"salah" + '
  '0.017*"aplikasi" + 0.015*"music" + 0.015*"match" + 0.012*"kesal" + '
  '0.012*"pakai" + 0.010*"kasih"'),
 (1,
  '0.046*"lengkap" + 0.046*"saksi" + 0.043*"indosiar" + 0.037*"special" + '
  '0.033*"ramadhan" + 0.033*"sore" + 0.033*"pkl" + 0.033*"wib" + 0.017*"pintu" '
  '+ 0.017*"berkah"'),
 (2,
  '0.049*"indonesia" + 0.041*"vs" + 0.040*"tayang" + 0.038*"cup" + '
  '0.031*"thomas" + 0.030*"match" + 0.028*"uber" + 0.027*"vidio" + 0.023*"kak" '
  '+ 0.021*"jam"'),
 (3,
  '0.101*"ka" + 0.056*"terima" + 0.052*"kasih" + 0.045*"mr" + 0.030*"dm" + '
  '0.022*"oke" + 0.019*"tayang" + 0.017*"balas" + 0.017*"langgan" + '
  '0.016*"vidio"'),
 (4,
  '0.036*"nonton" + 0.024*"tanding" + 0.023*"selesai" + 0.022*"kak" + '
  '0.021*"vs" + 0.020*"match" + 0.019*"chelsea" + 0.018*"cup" + 0.018*"fa" + '
  '0.018*"city"'),
 (5,
  '0.096*"paket" + 0.063*"platinum" + 0.055*"langgan" + 0.044*"beli" + '
  '0.030*"spotv" + 0.029*"nonton"